In [2]:
import numpy as np
import pandas as pd
from script.data_pipeline import shinyDataFetcher
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.regression.rolling import RollingOLS
import matplotlib.pyplot as plt
import itertools
import warnings
import statsmodels.tsa.stattools as ts
from IPython.display import HTML, display
import itables

warnings.filterwarnings("ignore")

def write_table(df, filename):
    html = itables.to_html_datatable(df, display_logo_when_loading=True)
    with open(f'../docs/dataframe/{filename}.html', 'w') as f:
        f.write(html)

EV_stocks = ["TSLA", "RIVN", "LCID", "F", "GM", "NIO", "XPEV", "BYDDF", "LI"]
EV_stocks_data = {}
for stock in EV_stocks:
    EV_stocks_data[stock] = shinyDataFetcher(asset=stock, durationStr="3 Y", barSizeSetting="1 day").fetch_asset_data()

log_prices = {}
for ticker in EV_stocks:
    df_temp = EV_stocks_data[ticker].copy()
    df_temp["log_close"] = np.log(df_temp["close"])
    log_prices[ticker] = df_temp.set_index("timestamp")["log_close"].dropna()



In [64]:
EV_stocks

['TSLA', 'RIVN', 'LCID', 'F', 'GM', 'NIO', 'XPEV', 'BYDDF', 'LI']

In [ ]:
# Loop over all unique pairs
for s1, s2 in itertools.combinations(EV_stocks, 2):
    # Align datasets by timestamp
    print(f"Processing pair: {s1} & {s2}")
    df_pair = pd.concat([log_prices[s1], log_prices[s2]], axis=1).dropna()
    if df_pair.empty:
        continue

    X = log_prices[s1]
    Y = log_prices[s2]
    X_const = sm.add_constant(X)  # Adds a constant term to the predictor
    model = sm.OLS(Y, X_const).fit()
    print(f"OLS Coefficients: {model.params.to_dict()}, OLS p-value: {model.pvalues[1]:.4f}, OLS R-squared: {model.rsquared:.4f}")

    # Perform cointegration test
    result = coint(df_pair.iloc[:, 0], df_pair.iloc[:, 1])
    test_stat, p_value, crit_values = result

    print(f"Test Statistic: {test_stat:.3f}, p-value: {p_value:.4f}", f"Critical Values: {crit_values}")
    print(f"r-square > 0.5? {'Yes' if model.rsquared > 0.5 else 'No'}")
    print(f"Is cointegrated? {'Yes' if p_value < 0.05 else 'No'}")
    print("-" * 50)

In [4]:
fig = go.Figure()
asset_a = "TSLA"
asset_b = "RIVN"
fig.add_trace(go.Scatter(x=EV_stocks_data[asset_a]["timestamp"], y=np.log(EV_stocks_data[asset_a]["close"]), mode="lines", name=f"{asset_a}"))
fig.add_trace(go.Scatter(x=EV_stocks_data[asset_b]["timestamp"], y=np.log(EV_stocks_data[asset_b]["close"]), mode="lines", name=f"{asset_b}"))
fig.update_layout(title=f"{asset_a}  vs. {asset_b} Stock Log Prices", xaxis_title="timestamp", yaxis_title="Price (USD)", legend=dict(x=0, y=1))
fig.show()
fig.write_html("html_plot/log_price.html")

In [ ]:
X = np.log(EV_stocks_data[asset_a]["close"])
Y = np.log(EV_stocks_data[asset_b]["close"])
X_const = sm.add_constant(X)  # Adds a constant term to the predictor
model = sm.OLS(Y, X_const).fit()
print(model.summary())


result = ts.coint(X, Y)
print("Cointegration test result:")
print("========================================")
print("Test Statistic:", result[0])
print("p-value:", result[1])

                            OLS Regression Results                            
Dep. Variable:                  close   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     22.17
Date:                Thu, 17 Apr 2025   Prob (F-statistic):           2.97e-06
Time:                        10:13:11   Log-Likelihood:                -371.88
No. Observations:                 753   AIC:                             747.8
Df Residuals:                     751   BIC:                             757.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4184      0.305      4.652      0.0

In [3]:
def rolling_adf(spread_series, time_index, window=90):
    adf_stats = []
    p_values = []
    dates = []

    for i in range(window, len(spread_series)):
        window_data = spread_series.iloc[i - window : i]
        result = adfuller(window_data, maxlag=1, autolag=None)
        adf_stats.append(result[0])  # Test statistic
        p_values.append(result[1])  # p-value
        dates.append(time_index.iloc[i])
    return pd.DataFrame({"timestamp": dates, "adf_stat": adf_stats, "p_value": p_values})


trade_df = pd.merge(EV_stocks_data["RIVN"][["timestamp", "open", "close"]], EV_stocks_data["TSLA"][["timestamp", "open", "close"]], on="timestamp", suffixes=("_RIVN", "_TSLA"))
trade_df["log_close_TSLA"] = np.log(trade_df["close_TSLA"])
trade_df["log_close_RIVN"] = np.log(trade_df["close_RIVN"])
X = trade_df["log_close_TSLA"].astype(float)
Y = trade_df["log_close_RIVN"].astype(float)
X_with_const = sm.add_constant(X)
model = RollingOLS(endog=Y, exog=X_with_const, window=20)
rres = model.fit()
trade_df["hedge_ratio"] = rres.params["log_close_TSLA"]
trade_df["spread"] = trade_df["log_close_RIVN"] - trade_df["hedge_ratio"] * trade_df["log_close_TSLA"]
trade_df["z_score"] = (trade_df["spread"] - trade_df["spread"].rolling(window=20).mean()) / trade_df["spread"].rolling(window=20).std()
trade_df.dropna(inplace=True)
rolling_adf_df = rolling_adf(trade_df["spread"], trade_df["timestamp"], window=60)
trade_df = pd.merge(trade_df, rolling_adf_df, on="timestamp", how="left")
trade_df.dropna(inplace=True)


In [43]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=trade_df["timestamp"], y=trade_df["spread"], mode="lines", name=f"spread"))
fig.add_trace(go.Scatter(x=trade_df["timestamp"], y=trade_df["z_score"], mode="lines", name=f"z-score", opacity=0.4))
fig.add_hline(y=2, line_dash="dash", line_color="red", annotation_text="z-score upper threshold", annotation_position="top right", opacity=0.4)
fig.add_hline(y=-2, line_dash="dash", line_color="red", annotation_text="z-score lower threshold", annotation_position="bottom right", opacity=0.4)
fig.update_layout(title="spread through time")
fig.show()
# fig.write_html("html_plot/log_price.html")

In [7]:
fig = go.Figure()
rolling_window = 60
fig.add_trace(go.Scatter(x=rolling_adf_df["timestamp"], y=rolling_adf_df["p_value"], mode="lines", name="Rolling ADF p-value"))
fig.add_hline(y=0.05, line_dash="dash", line_color="red", annotation_text="Significance threshold (0.05)", annotation_position="bottom right")
fig.update_layout(title=f"Rolling ADF Test on Spread (window={rolling_window})", xaxis_title="timestamp", yaxis_title="p-value")
fig.show()
fig.write_html('html_plot/adf_test.html')

In [42]:
df = trade_df.copy()
df["signal_z_score"] = df["z_score"].shift(1)
df["signal_spread"] = df["spread"].shift(1)
df["signal_p_value"] = df["p_value"].shift(1)
df["position"] = 0
df["trade_entry"] = False
df["trade_exit"] = False
df["pnl"] = 0.0



in_position = False
entry_index = None
entry_tsla_price = None
entry_rivn_price = None
tsla_qty = 0
rivn_qty = 0



blotter = []
ledger = []


notional = 10000
account_value = 100000
cumulative_pnl = 0.0

tight_threshold = 1
divergence_target = 2.5 
max_holding_period = 20

for i in range(1, len(df)):
    z = df["signal_z_score"].iloc[i]
    spread = df["signal_spread"].iloc[i]
    p_value = df["signal_p_value"].iloc[i]
    tsla_open = df["open_TSLA"].iloc[i]
    rivn_open = df["open_RIVN"].iloc[i]
    tsla_close = df["close_TSLA"].iloc[i]
    rivn_close = df["close_RIVN"].iloc[i]
    timestamp = df["timestamp"].iloc[i]
    beta = df["hedge_ratio"].iloc[i]

    if not in_position and p_value < 0.05:
        if abs(z) < tight_threshold:
            # Default Rule: Long TSLA, Short RIVN
            tsla_qty = round(notional / tsla_open)
            rivn_qty = -round((abs(beta) * notional) / rivn_open)
            direction = "long TSLA, short RIVN (divergence bet)"

            # Record Entry
            df.at[df.index[i], "trade_entry"] = True
            in_position = True
            entry_index = i
            entry_tsla_price = tsla_open
            entry_rivn_price = rivn_open
            blotter.append(
                {
                    "entry_time": timestamp,
                    "entry_tsla": tsla_open,
                    "entry_rivn": rivn_open,
                    "direction": direction,
                    "entry_spread": spread,
                    "hedge_ratio": beta,
                    "TSLA_qty": tsla_qty,
                    "RIVN_qty": rivn_qty,
                }
            )


    daily_pnl = 0.0
    if in_position:
        daily_pnl = (tsla_close - entry_tsla_price) * tsla_qty + (rivn_close - entry_rivn_price) * rivn_qty
        df.at[df.index[i], "pnl"] = daily_pnl
        # exit condition
        mean_reversion_triggered = abs(z) > divergence_target
        timeout_triggered = i - entry_index >= max_holding_period and (abs(z) < 1 and p_value < 0.05)
        if mean_reversion_triggered or timeout_triggered:
            reason = "divergence_target_hit" if mean_reversion_triggered else "timeout"
            df.at[df.index[i], "position"] = 0
            df.at[df.index[i], "trade_exit"] = True
            
            pnl_tsla = (tsla_open - entry_tsla_price) * tsla_qty
            pnl_rivn = (rivn_open - entry_rivn_price) * rivn_qty
            daily_pnl = pnl_tsla + pnl_rivn
            
            df.at[df.index[i], "pnl"] = daily_pnl
            cumulative_pnl += daily_pnl
            account_value += daily_pnl
            
            blotter[-1].update({"exit_time": timestamp, "exit_tsla": tsla_open, "exit_rivn": rivn_open, "exit_spread": df["spread"].iloc[i], "pnl": daily_pnl, "reason": reason})
            in_position = False
            entry_index = None
            entry_tsla_price = None
            entry_rivn_price = None
            tsla_qty = 0
            rivn_qty = 0
    else:
        df.at[df.index[i], "pnl"] = 0.0

    ledger.append(
        {
            "timestamp": timestamp,
            "TSLA_qty": tsla_qty,
            "RIVN_qty": rivn_qty,
            "TSLA_close": tsla_close,
            "RIVN_close": rivn_close,
            "daily_unrealized_pnl": daily_pnl,
            "account_value": account_value + daily_pnl,
            "realized_pnl": cumulative_pnl,
        }
    )



blotter_df = pd.DataFrame(blotter)
ledger_df = pd.DataFrame(ledger)

In [43]:
blotter_df
write_table(blotter_df, 'blotter')

In [44]:
blotter_df

,entry_time,entry_tsla,entry_rivn,direction,entry_spread,hedge_ratio,TSLA_qty,RIVN_qty,exit_time,exit_tsla,exit_rivn,exit_spread,pnl,reason
0,2022-12-15,153.41,24.03,"long TSLA, short RIVN (divergence bet)",-2.833855,1.194600,65,-497,2023-01-17,125.70,16.68,-1.343542,1851.80,timeout
1,2023-01-31,164.57,18.10,"long TSLA, short RIVN (divergence bet)",1.466066,0.313163,61,-173,2023-03-01,206.33,17.65,3.228720,2625.21,divergence_target_hit
2,2023-05-24,182.23,13.96,"long TSLA, short RIVN (divergence bet)",-1.208031,0.705615,55,-505,2023-07-05,278.77,20.70,-3.851991,1906.00,divergence_target_hit
3,2023-08-09,250.83,24.80,"long TSLA, short RIVN (divergence bet)",4.553909,0.026105,40,-11,2023-09-07,244.91,22.47,-1.239799,-211.17,timeout
4,2023-09-08,251.22,23.40,"long TSLA, short RIVN (divergence bet)",-1.239799,0.821496,40,-351,2023-10-06,253.98,17.73,0.296465,2100.57,timeout
5,2023-10-09,255.33,18.30,"long TSLA, short RIVN (divergence bet)",0.296465,0.397916,39,-217,2023-10-13,258.88,19.27,7.054522,-72.04,divergence_target_hit
6,2023-10-23,209.99,16.55,"long TSLA, short RIVN (divergence bet)",-0.483207,0.748157,48,-452,2023-11-17,231.97,16.48,0.828442,1086.68,divergence_target_hit
7,2023-12-04,235.75,17.79,"long TSLA, short RIVN (divergence bet)",0.963576,0.479619,42,-270,2023-12-12,238.36,18.91,-5.827830,-192.78,divergence_target_hit
8,2024-04-11,172.59,10.25,"long TSLA, short RIVN (divergence bet)",0.706461,0.260430,58,-254,2024-04-16,156.77,8.38,-6.844699,-442.58,divergence_target_hit
9,2024-05-21,175.50,10.24,"long TSLA, short RIVN (divergence bet)",0.468917,0.254302,57,-248,2024-05-23,181.80,10.57,3.966660,277.26,divergence_target_hit


In [51]:
ledger_df
write_table(ledger_df, 'ledger')

In [52]:
itables.show(ledger_df)

In [50]:
import numpy as np
import pandas as pd

# First, calculate gross_exposure and simple return per trade
def compute_trade_return(row):
    gross_exposure = abs(row["TSLA_qty"] * row["entry_tsla"]) + abs(row["RIVN_qty"] * row["entry_rivn"])
    return row["pnl"] / gross_exposure

blotter_df["return"] = blotter_df.apply(compute_trade_return, axis=1)

# Ensure datetime parsing
blotter_df["entry_time"] = pd.to_datetime(blotter_df["entry_time"])
blotter_df["exit_time"] = pd.to_datetime(blotter_df["exit_time"])

# Total trading duration
years_active = 3  # You mentioned using 3 years

# Basic stats
returns = blotter_df["return"]
average_return = returns.mean()
volatility = returns.std()
geo_mean = np.exp(np.log1p(returns).mean()) - 1  # Equivalent to geometric mean of (1 + return) - 1

# Sharpe Ratio (assumes 0% risk-free rate)
sharpe_ratio = average_return / volatility if volatility > 0 else np.nan

# Number of trades
num_trades = len(blotter_df)
avg_trades_per_year = num_trades / years_active

# Calculate average holding duration in days
blotter_df["holding_days"] = (blotter_df["exit_time"] - blotter_df["entry_time"]).dt.days
avg_days_per_trade = blotter_df["holding_days"].mean()

# Annualization factor
annual_factor = 252 / avg_days_per_trade

annualized_return = (1 + geo_mean)**annual_factor - 1
annualized_volatility = volatility * np.sqrt(annual_factor)
annualized_sharpe = annualized_return / annualized_volatility

# Print all stats
stats = {
    "Number of Trades": num_trades,
    "Average Return per Trade (%)": round(average_return * 100, 4),
    "Volatility of Returns (%)": round(volatility * 100, 4),
    "Geometric Mean Return per Trade (%)": round(geo_mean * 100, 4),
    "Sharpe Ratio (0% RF)": round(sharpe_ratio, 4),
    "Average Trades per Year": round(avg_trades_per_year, 2),
    "Average Holding Duration (days)": round(avg_days_per_trade, 2),
    "Annualized Return (%)": round(annualized_return * 100, 4),
    "Annualized Volatility (%)": round(annualized_volatility * 100, 4),
    "Annualized Sharpe Ratio": round(annualized_sharpe, 4),
    "Total PnL": round(blotter_df["pnl"].sum(), 2),
}

for k, v in stats.items():
    print(f"{k}: {v}")


Number of Trades: 14
Average Return per Trade (%): 6.7905
Volatility of Returns (%): 15.3163
Geometric Mean Return per Trade (%): 5.9258
Sharpe Ratio (0% RF): 0.4434
Average Trades per Year: 4.67
Average Holding Duration (days): 26.93
Annualized Return (%): 71.3838
Annualized Volatility (%): 46.8542
Annualized Sharpe Ratio: 1.5235
Total PnL: 12902.38


In [37]:
# Plot cumulative PnL with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=ledger_df.index, y=ledger_df["account_value"], mode="lines", name="account value"))
fig.update_layout(title="NAV Over Time", xaxis_title="Date", yaxis_title="account value")
fig.show()
fig.write_html("../docs/images/cumulative_pnl.html")

In [17]:
import pprint
pprint.pprint(summary)

{'Average Holding Period (days)': np.float64(22.21),
 'Average Trade Return': np.float64(527.3945),
 'Losing Trades': np.int64(89),
 'Total PnL': np.float64(125519.89),
 'Total Trades': np.int64(14),
 'Win Rate (%)': np.float64(1064.29),
 'Winning Trades': np.int64(149)}


In [53]:
import plotly.express as px


def calc_rtns(row):
    gross_exposure = abs(row["RIVN_qty"] * row["entry_rivn"]) + abs(row["TSLA_qty"] * row["entry_tsla"])
    row["trade_rtn"] = row["pnl"] / gross_exposure
    row["bench_tsla_rtn"] = (row["exit_tsla"] - row["entry_tsla"]) / row["entry_tsla"]
    row["bench_rivn_rtn"] = (row["entry_rivn"] - row["exit_rivn"]) / row["entry_rivn"]   # For short
    return row


ab_self_data = blotter_df.dropna().apply(calc_rtns, axis=1)[["trade_rtn", "bench_tsla_rtn", "bench_rivn_rtn"]]

for symbol in ["tsla", "rivn"]:

    ab_self_fig = px.scatter(ab_self_data * 100, x=f"bench_{symbol}_rtn", y="trade_rtn", trendline="ols", title="Strategy Realized Returns Sensitivity wrt the Underlying:" + symbol.upper() + "<br>")
    overall = px.get_trendline_results(ab_self_fig).iloc[0]["px_fit_results"].params

    ab_self_fig = (
        ab_self_fig.add_annotation(
            text="<b>Overall</b>: alpha=" + str(round(overall[0], 3)) + "%; beta=" + str(round(overall[1], 3)),
            xref="paper",
            yref="paper",
            x=0.05,
            y=1.17,
            showarrow=False,
            font=dict(size=15, color="orange"),
        )
        .update_layout(
            yaxis=dict(tickfont=dict(size=12), tickformat=".3s", showgrid=False, title=dict(text="Strategy Return per trading period", font=dict(size=15))),
            xaxis=dict(
                tickfont=dict(size=12),
                tickformat=".3s",
                showgrid=False,
                title=dict(text="Underlying Asset Return, Same Timestamps", font=dict(size=15)),
            ),
            yaxis_ticksuffix="%",
            xaxis_ticksuffix="%",
        )
        .update_traces(marker=dict(size=10))
    )
    ab_self_fig.show()
    ab_self_fig.write_html(f"../docs/images/Return_sensitivity_{symbol.upper()}.html")

In [39]:
blotter_df.columns

Index(['entry_time', 'entry_tsla', 'entry_rivn', 'direction', 'entry_spread',
       'hedge_ratio', 'TSLA_qty', 'RIVN_qty', 'exit_time', 'exit_tsla',
       'exit_rivn', 'exit_spread', 'pnl', 'reason'],
      dtype='object')